In [ ]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

seattle_graph = ox.graph_from_place(
    "Seattle, Washington, USA",
    network_type="walk"
)
seattle_graph = ox.project_graph(seattle_graph)

ox.save_graphml(seattle_graph, "seattle.graphml")

print("Graph downloaded!")

In [ ]:
nodes, edges =  ox.graph_to_gdfs(seattle_graph)

In [ ]:
df= pd.read_csv('seattle_dataset.csv')
df['geometry'] = df.apply(lambda row: Point(row['longitude'],row['latitude']), axis=1)
df.head()

In [ ]:
barriers = gpd.GeoDataFrame(df, geometry="geometry",crs = "EPSG:4326")
barriers = barriers.to_crs(edges.crs)
barriers

,Unnamed: 0,type,geometry_type,attribute_id,label_type,neighborhood,severity,is_temporary,longitude,latitude,geometry
0,0,Feature,Point,52096165,SurfaceProblem,Atlantic,4.0,False,-122.298981,47.594616,POINT (-122.29898 47.59462)
1,1,Feature,Point,52096166,SurfaceProblem,Atlantic,3.0,False,-122.301071,47.593357,POINT (-122.30107 47.59336)
2,2,Feature,Point,52096167,SurfaceProblem,Atlantic,4.0,False,-122.301079,47.596844,POINT (-122.30108 47.59684)
3,3,Feature,Point,52096168,SurfaceProblem,Atlantic,4.0,False,-122.301071,47.596500,POINT (-122.30107 47.5965)
4,4,Feature,Point,52096365,NoCurbRamp,Atlantic,4.0,False,-122.306274,47.599930,POINT (-122.30627 47.59993)
...,...,...,...,...,...,...,...,...,...,...,...
81968,81968,Feature,Point,52096160,SurfaceProblem,Atlantic,3.0,False,-122.310753,47.601601,POINT (-122.31075 47.6016)
81969,81969,Feature,Point,52096161,SurfaceProblem,Atlantic,3.0,False,-122.297600,47.597965,POINT (-122.2976 47.59797)
81970,81970,Feature,Point,52096162,SurfaceProblem,Atlantic,3.0,False,-122.298042,47.599327,POINT (-122.29804 47.59933)
81971,81971,Feature,Point,52096163,SurfaceProblem,Atlantic,4.0,False,-122.305794,47.596718,POINT (-122.30579 47.59672)


In [ ]:
barriers_edges = gpd.sjoin_nearest(
    barriers,
    edges,
    how="left",
    distance_col="dist",
    max_distance=20
)

In [ ]:
barriers_edges = barriers_edges.sort_values("dist").drop_duplicates(subset="Unnamed: 0", keep='first')
barriers_edges

,Unnamed: 0,type,geometry_type,attribute_id,label_type,neighborhood,severity,is_temporary,longitude,latitude,...,name,bridge,service,width,tunnel,access,junction,ref,est_width,dist
49151,49151,Feature,Point,52148537,CurbRamp,Lawton Park,1.0,False,-122.403496,47.658451,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000010
63012,63012,Feature,Point,52081996,CurbRamp,Broadway,1.0,False,-122.328850,47.613613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000055
32692,32692,Feature,Point,52135646,NoSidewalk,East Queen Anne,4.0,False,-122.352974,47.640766,...,Lorentz Place North,NaN,driveway,NaN,NaN,NaN,NaN,NaN,NaN,0.000105
74606,74606,Feature,Point,52166896,CurbRamp,Minor,2.0,False,-122.308922,47.614155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000142
49888,49888,Feature,Point,52149721,NoCurbRamp,Lawton Park,3.0,False,-122.394600,47.654144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80630,80630,Feature,Point,52094046,NoSidewalk,Bryant,3.0,False,-122.289185,47.691681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80632,80632,Feature,Point,52094048,NoSidewalk,Bryant,3.0,False,-122.289047,47.691677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80634,80634,Feature,Point,52094050,NoSidewalk,Bryant,3.0,False,-122.289139,47.691597,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80636,80636,Feature,Point,52094052,NoSidewalk,Bryant,5.0,False,-122.281235,47.673996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
edge_features = barriers_edges.groupby(["u","v","key"]).agg({
    "severity": ["mean","max","count"],
    "is_temporary": "mean",
    "length": "first"
})

In [ ]:
edge_features.rename(columns={'mean':'avg_severity', 'max': 'max_severity','count':'num_barriers'}, level=1,inplace=True)
edge_features.drop('is_temporary', axis=1, inplace=True)
edge_features.reset_index(inplace=True)

/var/folders/4c/hyv6qz1537z47z1zpjs7r47r0000gn/T/ipykernel_21830/3653444784.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_features.drop('is_temporary', axis=1, inplace=True)


In [134]:
edge_features.columns = [
    "".join(col).strip("") if isinstance(col, tuple) else col
    for col in edge_features.columns
]
print(edge_features.columns)

Index(['u', 'v', 'key', 'severityavg_severity', 'severitymax_severity',
       'severitynum_barriers', 'lengthfirst'],
      dtype='object')


In [ ]:
label_counts = (
    barriers_edges
    .groupby(["u","v","key","label_type"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

In [ ]:
label_stats = (
    barriers_edges
    .groupby(["u","v","key","label_type"])
    .agg(
        count=("severity","count"),
        avg=("severity","mean"),
        max=("severity","max")
    )
    .reset_index()
)

In [170]:
label_stats_wide = (
    label_stats
    .set_index(["u","v","key","label_type"])
    .unstack(fill_value=0)
)

In [172]:
label_stats_wide.columns = [
    f"{label}_{metric}"
    for metric, label in label_stats_wide.columns
]

label_stats_wide = label_stats_wide.reset_index()
label_stats_wide

,u,v,key,CurbRamp_count,NoCurbRamp_count,NoSidewalk_count,Obstacle_count,Occlusion_count,Other_count,SurfaceProblem_count,...,Occlusion_avg,Other_avg,SurfaceProblem_avg,CurbRamp_max,NoCurbRamp_max,NoSidewalk_max,Obstacle_max,Occlusion_max,Other_max,SurfaceProblem_max
0,2.954543e+07,8.818370e+09,0.0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.993750e+07,6.292268e+09,0.0,2,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.993752e+07,1.327758e+10,0.0,2,0,0,0,0,0,0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.993752e+07,6.292222e+09,0.0,2,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.993759e+07,4.694839e+09,0.0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47612,1.352070e+10,1.352070e+10,0.0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
47613,1.352073e+10,6.582146e+09,0.0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
47614,1.352073e+10,1.352073e+10,0.0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
47615,1.352481e+10,1.169558e+10,0.0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
edge_features_with_labels = edge_features.merge(
    label_stats_wide,
    on=["u","v","key"],
    how="left"
)

edge_features_with_labels = edge_features_with_labels.fillna(0)
edge_features_with_labels

,u,v,key,severityavg_severity,severitymax_severity,severitynum_barriers,lengthfirst,CurbRamp_count,NoCurbRamp_count,NoSidewalk_count,...,Occlusion_avg,Other_avg,SurfaceProblem_avg,CurbRamp_max,NoCurbRamp_max,NoSidewalk_max,Obstacle_max,Occlusion_max,Other_max,SurfaceProblem_max
0,2.954543e+07,8.818370e+09,0.0,1.0,1.0,1,317.313855,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.993750e+07,6.292268e+09,0.0,1.0,1.0,2,19.772566,2,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.993752e+07,1.327758e+10,0.0,2.0,3.0,2,13.534974,2,0,0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.993752e+07,6.292222e+09,0.0,1.0,1.0,2,16.156787,2,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.993759e+07,4.694839e+09,0.0,3.0,3.0,1,11.729261,1,0,0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47612,1.352070e+10,1.352070e+10,0.0,3.0,3.0,1,15.380775,0,1,0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
47613,1.352073e+10,6.582146e+09,0.0,1.0,1.0,1,14.632312,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
47614,1.352073e+10,1.352073e+10,0.0,3.0,3.0,1,6.657347,0,0,1,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
47615,1.352481e+10,1.169558e+10,0.0,1.0,1.0,1,26.884812,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
nodes_ll = nodes.to_crs(epsg=4326)

edge_features_with_labels["u_lat"] = edge_features_with_labels["u"].apply(lambda x: nodes_ll.loc[x].geometry.y)
edge_features_with_labels["u_lon"] = edge_features_with_labels["u"].apply(lambda x: nodes_ll.loc[x].geometry.x)

edge_features_with_labels["v_lat"] = edge_features_with_labels["v"].apply(lambda x: nodes_ll.loc[x].geometry.y)
edge_features_with_labels["v_lon"] = edge_features_with_labels["v"].apply(lambda x: nodes_ll.loc[x].geometry.x)


In [ ]:
edge_features_with_labels.drop(['u','v'],axis=1, inplace=True)
edge_features_with_labels.columns

In [ ]:
neighborhood_geo_data = gpd.read_file("seattle_neighborhoods.geojson").to_crs("EPSG:4326")
gdf = gpd.GeoDataFrame(
    edge_features_with_labels,
    geometry=gpd.points_from_xy(edge_features_with_labels["u_lon"], edge_features_with_labels["u_lat"]),
    crs="EPSG:4326"
)
# spatial join
edge_features_with_neighborhood = gpd.sjoin(gdf, neighborhood_geo_data, how="left", predicate="within")
edge_features_with_neighborhood["neighborhood"] = edge_features_with_neighborhood["name"]   
edge_features_with_neighborhood = edge_features_with_neighborhood.drop(columns=["geometry", "index_right"])


In [ ]:
edge_features_with_labels_neighborhood = edge_features_with_neighborhood[['severityavg_severity', 'severitymax_severity',
       'severitynum_barriers', 'lengthfirst', 'CurbRamp_count',
       'NoCurbRamp_count', 'NoSidewalk_count', 'Obstacle_count',
       'Occlusion_count', 'Other_count', 'SurfaceProblem_count',
       'CurbRamp_avg', 'NoCurbRamp_avg', 'NoSidewalk_avg', 'Obstacle_avg',
       'Occlusion_avg', 'Other_avg', 'SurfaceProblem_avg', 'CurbRamp_max',
       'NoCurbRamp_max', 'NoSidewalk_max', 'Obstacle_max', 'Occlusion_max',
       'Other_max', 'SurfaceProblem_max', 'u_lat', 'u_lon', 'v_lat', 'v_lon',
       'nhood']]
edge_features_with_labels_neighborhood.to_csv("gnn_input.csv")